In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sportsipy.ncaab import boxscore
from sportsipy.ncaab.teams import Teams
from sportsipy.ncaab.teams import Team

In [2]:
# TODO: this will be where we loop through all files and get the CSV as a new dataframe
directory = '../data/team_csvs/2019-2020/'
for filename in os.scandir(directory):
    # create DF from CSV 
    try: 
        teamschedule_df = pd.read_csv(filename)

        #filename variable is an nt.DirEntry type. We need to convert it to string
        file_name = str(filename)
        print(file_name)
        # <DirEntry '109_Georgia_season_boxscores_2019.csv'>
        csvname = file_name[11:-2]
        print(csvname)

        # Now, we need to slice the string in a sustainable way. We'll slice out everything up to and including the first underscore
        file_name = file_name.split('_', 1)
        print(file_name)
        # ["<DirEntry '109", "Georgia_season_boxscores_2019.csv'>"]


        file_name = file_name[1]
        print(file_name)
        # Georgia_season_boxscores_2019.csv'>


        #next we extract team name and season - we know that everything after the name is constant for all files
        team_name = file_name[:-28]
        season = file_name[-10:-6]
        print(team_name)
        print(season)
        print('getting team abbreviation')
        for team in Teams(season):
            if team.name == team_name:
                abbrev = team.abbreviation
                break
            
        print (abbrev)
        teamschedule_df.rename(columns={'Unnamed: 0': 'boxscore'}, inplace=True)
        teamschedule_df['game_number'] = teamschedule_df.index

        # Conditional column - we have a pair of criteria that tell us if a given game is home based on the winner of the game (home vs. away) and 
        # the abbreviation of the winning team
        teamschedule_df['{}_homegame'.format(abbrev)] = (teamschedule_df['winner'] == 'Home') & (teamschedule_df['winning_abbr'] == abbrev)
        teamschedule_df['{}_homegame2'.format(abbrev)] = (teamschedule_df['winner'] == 'Away') & (teamschedule_df['winning_abbr'] != abbrev)

        # set team_abbrev_home to True if either of the home game criteria that we ran above are true
        teamschedule_df['{}_home'.format(abbrev)] = (teamschedule_df['{}_homegame'.format(abbrev)] == True) | (teamschedule_df['{}_homegame2'.format(abbrev)] == True)

        # Make the game location (from perspective of the team) equal to Home if the team_abbrev_home column is True, otherwise make it away
        teamschedule_df['{}_location'.format(abbrev)] = teamschedule_df.apply(lambda x: 'Home' if x['{}_home'.format(abbrev)] == True else 'Away', axis=1)

        # Create a column with True/False based on whether the team won their game. This will be used later to calc win percentage
        teamschedule_df['team_win_bool'] = (teamschedule_df['winning_abbr'] == abbrev)

        # Set each game stat equal to the home team stat if they played at home, otherwise set to the away team stat
        teamschedule_df['total_threes_made'] = teamschedule_df.apply(lambda x: x['home_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_three_point_field_goals'], axis=1)
        teamschedule_df['total_2pt_made'] = teamschedule_df.apply(lambda x: x['home_two_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_two_point_field_goals'], axis=1)
        teamschedule_df['total_fg_attempts'] = teamschedule_df.apply(lambda x: x['home_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['away_field_goal_attempts'], axis=1)

        # Do the same thing for rebounds instead of shooting
        # rebounding percentage = team rebounds / total rebounds
        teamschedule_df['team_rebounds'] = teamschedule_df.apply(lambda x: x['home_total_rebounds'] if x['{}_home'.format(abbrev)] == True else x['away_total_rebounds'], axis=1)
        teamschedule_df['total_rebounds'] = teamschedule_df['home_total_rebounds'] + teamschedule_df['away_total_rebounds']

        # Do the same thing for defensive statistics - remember we need to invert this
        teamschedule_df['total_opp_threes_made'] = teamschedule_df.apply(lambda x: x['away_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['home_three_point_field_goals'], axis=1)
        teamschedule_df['total_opp_2pt_made'] = teamschedule_df.apply(lambda x: x['away_two_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['home_two_point_field_goals'], axis=1)
        teamschedule_df['total_opp_fg_attempts'] = teamschedule_df.apply(lambda x: x['away_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['home_field_goal_attempts'], axis=1)

        #finally, calculate the rolling win percentage
        # adding 1 to game number so that we can easily calculate the wins and losses
        teamschedule_df['team_wins'] = 0
        teamschedule_df['team_losses'] = 0
        teamschedule_df['team_wins'] = teamschedule_df.apply(lambda x: x['team_wins'] + 1 if x['team_win_bool'] == True else x['team_wins'], axis = 1)
        teamschedule_df['team_losses'] = teamschedule_df.apply(lambda x: x['team_losses'] + 1 if x['team_win_bool'] == False else x['team_losses'], axis = 1)

        #calculate the cumulative pregame stats heading in to each game
        teamschedule_df['pregame_total_threes_made'] = teamschedule_df.total_threes_made.cumsum() - teamschedule_df.total_threes_made
        teamschedule_df['pregame_total_2pt_made'] = teamschedule_df.total_2pt_made.cumsum() - teamschedule_df.total_2pt_made
        teamschedule_df['pregame_total_fg_attempts'] = teamschedule_df.total_fg_attempts.cumsum() - teamschedule_df.total_fg_attempts

        #calculate the actual target statistic which is eFG 
        teamschedule_df['pregame_eFG'] = (teamschedule_df.pregame_total_2pt_made + (teamschedule_df.pregame_total_threes_made * 1.5)) / teamschedule_df.pregame_total_fg_attempts

        #calculate the cumulative rebound percentage
        teamschedule_df['pregame_team_rebounds'] = teamschedule_df.team_rebounds.cumsum() - teamschedule_df.team_rebounds
        teamschedule_df['pregame_total_rebounds'] = teamschedule_df.total_rebounds.cumsum() - teamschedule_df.total_rebounds
        teamschedule_df['pregame_rebound_percentage'] = teamschedule_df.pregame_team_rebounds / teamschedule_df.pregame_total_rebounds


        #calculate the cumulative defensive statistic
        teamschedule_df['pregame_total_opp_threes_made'] = teamschedule_df.total_opp_threes_made.cumsum() - teamschedule_df.total_opp_threes_made
        teamschedule_df['pregame_total_opp_2pt_made'] = teamschedule_df.total_opp_2pt_made.cumsum() - teamschedule_df.total_opp_2pt_made
        teamschedule_df['pregame_total_opp_fg_attempts'] = teamschedule_df.total_opp_fg_attempts.cumsum() - teamschedule_df.total_opp_fg_attempts

        # calculate the defensive eFG - aka opponents eFG
        teamschedule_df['pregame_eFG_allowed'] = (teamschedule_df.pregame_total_opp_2pt_made + (teamschedule_df.pregame_total_opp_threes_made * 1.5)) / teamschedule_df.pregame_total_opp_fg_attempts

        #calculate the pregame win percentage - it's the total wins after the game, minus result of game (loss would have no impact on total wins)
        # minus total losses after game (win would have no impact) divided by the sum of the two
        teamschedule_df['pregame_win_percentage'] = (teamschedule_df.team_wins.cumsum() - teamschedule_df.team_wins) / ((teamschedule_df.team_wins.cumsum() - teamschedule_df.team_wins) + (teamschedule_df.team_losses.cumsum() - teamschedule_df.team_losses))



        print('dataframe complete')

        # Overwrite the CSV with the new dataframe containing stats
        teamschedule_df.to_csv('../data/team_csvs/2019-2020/{}'.format(csvname))
        print(team_name, ' CSV complete')
    except:
        print(team_name, 'error')



<DirEntry "'101_Furman_season_boxscores_2019.csv">
'101_Furman_season_boxscores_2019.csv
['<DirEntry "\'101', 'Furman_season_boxscores_2019.csv">']
Furman_season_boxscores_2019.csv">
Furman
2019
getting team abbreviation
FURMAN
dataframe complete
Furman  CSV complete
<DirEntry '100_Fresno State_season_boxscores_2019.csv'>
100_Fresno State_season_boxscores_2019.csv
["<DirEntry '100", "Fresno State_season_boxscores_2019.csv'>"]
Fresno State_season_boxscores_2019.csv'>
Fresno State
2019
getting team abbreviation
FRESNO-STATE
dataframe complete
Fresno State  CSV complete
<DirEntry '101_Furman_season_boxscores_2019.csv'>
101_Furman_season_boxscores_2019.csv
["<DirEntry '101", "Furman_season_boxscores_2019.csv'>"]
Furman_season_boxscores_2019.csv'>
Furman
2019
getting team abbreviation
FURMAN
dataframe complete
Furman  CSV complete
<DirEntry '102_Gardner-Webb_season_boxscores_2019.csv'>
102_Gardner-Webb_season_boxscores_2019.csv
["<DirEntry '102", "Gardner-Webb_season_boxscores_2019.csv'>"]


In [ ]:
#All of the below cell is to play around
#directory = '../data/team_csvs/2019-2020/'
#filename = '283_South Florida_season_boxscores_2019.csv'
#teamschedule_df = pd.read_csv(directory + filename)

#filename variable is an nt.DirEntry type. We need to convert it to string
#file_name = str(filename)
#print(file_name)

# Now, we need to slice the string in a sustainable way. We'll slice out everything up to and including the first underscore
#file_name = file_name.split('_', 1)
#print(file_name)

#file_name = file_name[1]
#print(file_name)
#next we extract team name and season - we know that everything after the name is constant for all files
#team_name = file_name[:-26]
#season = file_name[-8:-4]
#print(team_name)
#print(season)
#print('getting team abbreviation')

#for team in Teams(season):
#    if team.name == team_name:
#        abbrev = team.abbreviation
#        break

#print (abbrev)
#teamschedule_df.rename(columns={'Unnamed: 0': 'boxscore'}, inplace=True)
#teamschedule_df['game_number'] = teamschedule_df.index

# Conditional column - we have a pair of criteria that tell us if a given game is home based on the winner of the game (home vs. away) and 
# the abbreviation of the winning team
#teamschedule_df['{}_homegame'.format(abbrev)] = (teamschedule_df['winner'] == 'Home') & (teamschedule_df['winning_abbr'] == abbrev)
#teamschedule_df['{}_homegame2'.format(abbrev)] = (teamschedule_df['winner'] == 'Away') & (teamschedule_df['winning_abbr'] != abbrev)

# set team_abbrev_home to True if either of the home game criteria that we ran above are true
#teamschedule_df['{}_home'.format(abbrev)] = (teamschedule_df['{}_homegame'.format(abbrev)] == True) | (teamschedule_df['{}_homegame2'.format(abbrev)] == True)

# Make the game location (from perspective of the team) equal to Home if the team_abbrev_home column is True, otherwise make it away
#teamschedule_df['{}_location'.format(abbrev)] = teamschedule_df.apply(lambda x: 'Home' if x['{}_home'.format(abbrev)] == True else 'Away', axis=1)

# Set each game stat equal to the home team stat if they played at home, otherwise set to the away team stat
#teamschedule_df['total_threes_made'] = teamschedule_df.apply(lambda x: x['home_three_point_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_three_point_field_goals'], axis=1)
#teamschedule_df['total_fg_made'] = teamschedule_df.apply(lambda x: x['home_field_goals'] if x['{}_home'.format(abbrev)] == True else x['away_field_goals'], axis=1)
#teamschedule_df['total_fg_attempts'] = teamschedule_df.apply(lambda x: x['home_field_goal_attempts'] if x['{}_home'.format(abbrev)] == True else x['away_field_goal_attempts'], axis=1)

#calculate the cumulative pregame stats heading in to each game
#teamschedule_df['pregame_total_threes_made'] = teamschedule_df.total_threes_made.cumsum() - teamschedule_df.total_threes_made
#teamschedule_df['pregame_total_fg_made'] = teamschedule_df.total_fg_made.cumsum() - teamschedule_df.total_fg_made
#teamschedule_df['pregame_total_fg_attempts'] = teamschedule_df.total_fg_attempts.cumsum() - teamschedule_df.total_fg_attempts

#calculate the actual target statistic which is eFG 
#teamschedule_df['pregame_eFG'] = (teamschedule_df.pregame_total_fg_made + (teamschedule_df.pregame_total_threes_made / 2)) / teamschedule_df.pregame_total_fg_attempts
# (2PM + .5 * 3PM) / FGA
#print('dataframe complete')
#teamschedule_df.to_csv('../data/team_csvs/2019-2020/{}'.format(filename))
#print(team_name, ' CSV complete')



In [ ]:
teamschedule_df.head()

In [ ]:
teamschedule_df.head()

In [ ]:
teamschedule_df.head()

In [ ]:



#each team's csv should have that team's rolling stats in the CSV. Then, for each game in the giant boxscore DF, 
# I can go grab the team's pregame eFG for that particular game number. Would prob have to index on boxscore uri
